In [0]:
# coding:utf8
import warnings


class DefaultConfig(object):
    env = 'UNetResNet'  # visdom 环境
    model = 'UNetResNet'  # 使用的模型，名字必须与models/__init__.py中的名字一致
    ori_data_dir = '../input'
    train_data_images_root = '../input/train/images'  # 训练集存放路径
    train_data_masks_root = '../input/train/masks'  # 训练集mask存放路径
    test_data_root = '../input/test/images'  # 测试集存放路径
    load_model_path = None  # 加载预训练的模型的路径，为None代表不加载

    # 原始图片大小
    ori_image_h = 101
    ori_image_w = 101
    ori_image_channels = 3
    # 图片的输入
    image_h = 128
    image_w = 128
    image_channels = 3

    img_size_ori = 101
    img_size_target = 128

    batch_size = 32  # batch size
    use_gpu = True  # user GPU or not
    num_workers = 1  # how many workers for loading data
    print_freq = 20  # print info every N batch
    pretrained = True
    useValBestThred = True
    BestThred = 0.5
    valPercent = 0.2

    debug_file = '/tmp/debug'  # if os.path.exists(debug_file): enter ipdb
    result_file = 'result.csv'

    max_epoch = 50
    lr = 0.01  # initial learning rate
    lr_decay = 0.95  # when val_loss increase, lr = lr*lr_decay
    weight_decay = 0e-4  # 损失函数

    seed = 1126

    # Loss
    dice_weight =  5.0
    bce_weight = 1.0

    num_folds = 5
    stratified = True
    
    SIZE = 128
    PAD  = 16
    
    fold_iter_num = 1

def parse(self, kwargs):
    """
    根据字典kwargs 更新 config参数
    """
    for k, v in kwargs.items():
        if not hasattr(self, k):
            warnings.warn("Warning: opt has not attribut %s" % k)
        setattr(self, k, v)

    print('user config:')
    for k, v in self.__class__.__dict__.items():
        if not k.startswith('__'):
            print(k, getattr(self, k))

DefaultConfig.parse = parse
opt = DefaultConfig()

In [0]:
import numpy as np
import pydensecrf.densecrf as dcrf
from skimage.io import imread, imsave
from pydensecrf.utils import unary_from_labels, create_pairwise_bilateral
from skimage.color import gray2rgb
from skimage.color import rgb2gray
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import os


def rle_decode(rle_mask):
    '''
    rle_mask: run-length as string formated (start length)
    shape: (height,width) of array to return
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = rle_mask.split()
    starts, lengths = [np.asarray(x, dtype=int)
                                  for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(101*101, dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(101, 101)

# Original_image = Image which has to labelled
# Mask image = Which has been labelled by some technique..


def crf(original_image, mask_img):

    # Converting annotated image to RGB if it is Gray scale
    if(len(mask_img.shape) < 3):
        mask_img = gray2rgb(mask_img)

#     #Converting the annotations RGB color to single 32 bit integer
    annotated_label = mask_img[:, :, 0] + \
        (mask_img[:, :, 1] << 8) + (mask_img[:, :, 2] << 16)

#     # Convert the 32bit integer color to 0,1, 2, ... labels.
    colors, labels = np.unique(annotated_label, return_inverse=True)

    n_labels = 2

    # Setting up the CRF model
    d = dcrf.DenseCRF2D(
        original_image.shape[1], original_image.shape[0], n_labels)

    # get unary potentials (neg log probability)
    U = unary_from_labels(labels, n_labels, gt_prob=0.7, zero_unsure=False)
    d.setUnaryEnergy(U)

    # This adds the color-independent term, features are the locations only.
    d.addPairwiseGaussian(sxy=(3, 3), compat=3, kernel=dcrf.DIAG_KERNEL,
                      normalization=dcrf.NORMALIZE_SYMMETRIC)

    # Run Inference for 10 steps
    Q = d.inference(10)

    # Find out the most probable class for each pixel.
    MAP = np.argmax(Q, axis=0)

    return MAP.reshape((original_image.shape[0], original_image.shape[1]))


"""
used for converting the decoded image to rle mask

"""


def rle_encode(im):
    '''
    im: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = im.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

def crf_submission(submission,opt):
    for i in tqdm(range(submission.shape[0])):
        if str(submission.loc[i,'rle_mask'])!=str(np.nan):        
            decoded_mask = rle_decode(submission.loc[i,'rle_mask'])        
            orig_img = imread(os.path.join(opt.test_data_root,submission.loc[i,'id']+'.png'))        
            crf_output = crf(orig_img,decoded_mask)
            submission.loc[i,'rle_mask'] = rle_encode(crf_output)
    return submission


In [5]:
from google.colab import files
files.upload()

Saving submission.csv to submission.csv


{'submission.csv': b'id,rle_mask\nfe7c0c392e,5758 25 5859 42 5960 54 6061 63 6162 66 6263 69 6364 71 6465 72 6566 73 6667 74 6768 75 6869 76 6970 76 7071 76 7172 77 7273 77 7374 77 7475 78 7576 78 7677 79 7778 79 7879 80 7980 81 8081 83 8182 84 8283 86 8384 88 8485 90 8586 93 8687 97 8785 2 8788 1414\nd168218bac,\n28708ca79f,\n9327ed7fc3,\ncf19041b8e,60 23 161 23 262 23 363 23 464 23 565 22 665 23 766 22 867 21 968 20 1068 20 1169 19 1270 19 1370 19 1471 18 1572 17 1672 17 1773 16 1874 15 1974 14 2075 13 2176 12 2276 11 2376 10 2477 9 2577 8 2676 9 2775 10 2874 11 2973 12 3072 13 3171 14 3270 15 3370 15 3469 16 3569 17 3668 18 3768 18 3867 20 3967 20 4067 21 4167 22 4267 22 4367 23 4467 24 4567 24 4668 24 4768 24 4869 24 4969 25 5070 25 5170 25 5271 25 5372 24 5472 25 5573 24 5674 24 5774 24 5875 24 5976 24 6076 24 6177 24 6277 24 6378 24 6478 25 6579 24 6679 25 6779 25 6879 26 6979 26 7079 27 7179 28 7278 29 7376 31 7475 33 7576 32 7677 31 7778 31 7879 30 7980 29 8081 29 8182 28 8283 

In [9]:
import pandas as pd
submission = pd.read_csv('submission.csv')
submission.head()

,id,rle_mask
0,fe7c0c392e,5758 25 5859 42 5960 54 6061 63 6162 66 6263 6...
1,d168218bac,NaN
2,28708ca79f,NaN
3,9327ed7fc3,NaN
4,cf19041b8e,60 23 161 23 262 23 363 23 464 23 565 22 665 2...


In [14]:
submission_new = crf_submission(submission,opt)

  0%|          | 0/18000 [00:00<?, ?it/s]


FileNotFoundError: ignored